# R Aqr offsets calculations
Calculate x, y offsets for R Aqr target acquisition.

In [1]:
# Set up autoloader
%load_ext autoreload
%autoreload 2

In [2]:
# Set up modules and stuff
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

## Date range for JWST observation
This is the date range for the scheduled JWST observation

In [3]:
# JWST dates and times
JWST_TIME1 = '2024-10-30T18:19:00.000'    # Beginning of JWST observing window
JWST_TIME2 = '2024-12-12T00:19:55.000'    # Beginning of JWST observing window
JWST_TIME = '2024-11-06T18:19:00.000'     # Schediuled time JWST time

# Create Time-like objects with the JWST times
jwst_time1 = Time(JWST_TIME1, format='isot', scale='utc')
jwst_time2 = Time(JWST_TIME2, format='isot', scale='utc')
jwst_time = Time(JWST_TIME, format='isot', scale='utc')

## Reference coordinates for R Aqr and offset star
We provide two sets of coordinates for R Aqr (since the target had been observed twice), and the Gaia DR3 Coodinates of the offset star. The most recent set is the one actually used to calculate the updated coordinates at the time of the JWST observation. The earlier set of coordinates is used to test that the routine work properly (by using them as reference coordinates and checking that the result matches).

In [4]:
# R Aqr coordinates set 1
RA = '23:43:49.50120 hours'     # RA in sexagesimal format (ICRS)
DEC = '-15:17:04.804 degrees'   # DEC in sexagesimal format (ICRS) 
PMRA = 27.33                    # Proper motion in RA (mas/yr)
PMDEC = -30.37                  # Proper motion in DEC (mas/yr)
PARALLAX = 2.5931               # Parallax (mas)
OBS_DATE1 = '2020-10-01'        # Observation date
EPOCH = 2000.0                  # Reference epoch
FRAME = 'icrs'                  # Coordinate frame

# Create Skycoord object with the given coordinates
r_aqr1 = SkyCoord(RA,
                  DEC,
                  pm_ra_cosdec=PMRA*u.mas/u.yr,
                  pm_dec=PMDEC*u.mas/u.yr,
                  distance=Distance(parallax=PARALLAX*u.mas),
                  equinox=Time(EPOCH, format='jyear'),
                  obstime=Time(OBS_DATE1, format='iso', scale='utc'),
                  frame=FRAME
                 )
print(r_aqr1.obstime)

2020-10-01 00:00:00.000


In [5]:
# R Aqr coordinates set 2
RA = '23:43:49.50785 hours'     # RA in sexagesimal format (ICRS)
DEC = '-15:17:04.911 degrees'   # DEC in sexagesimal format (ICRS) 
PMRA = 27.33                    # Proper motion in RA (mas/yr)
PMDEC = -30.37                  # Proper motion in DEC (mas/yr)
PARALLAX = 2.5931               # Parallax (mas)
OBS_DATE2 = '2024-04-07'         # Observation date
EPOCH = 2000.0                  # Reference epoch
FRAME = 'icrs'                  # Coordinate frame

# Create Skycoord object with the given coordinates
r_aqr2 = SkyCoord(RA,
                  DEC,
                  pm_ra_cosdec=PMRA*u.mas/u.yr,
                  pm_dec=PMDEC*u.mas/u.yr,
                  distance=Distance(parallax=PARALLAX*u.mas),
                  equinox=Time(EPOCH, format='jyear'),
                  obstime=Time(OBS_DATE2, format='iso', scale='utc'),
                  frame=FRAME
                 )
print(r_aqr2)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (355.95628271, -15.2846975, 385.63881069)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (27.33, -30.37)>


In [6]:
# Gaia DR3 2419576423272148480 coordinates
RA = '355.94560363551375 degrees'    # RA in decimal degrees (ICRS)
DEC = '-15.274577751934695 degrees'  # DEC in decimal degrees (ICRS) 
PMRA = -2.708690975158548            # Proper motion in RA (mas/yr)
PMDEC = -4.0809858570655875          # Proper motion in DEC (mas/yr)
OBS_DATE = 2016.0                    # Observation date
EPOCH = 2016.0                       # Reference epoch
FRAME = 'icrs'                       # Coordinate frame

# Create Skycoord object with the given coordinates
acqt = SkyCoord(RA,
                DEC,
                pm_ra_cosdec=PMRA*u.mas/u.yr,
                pm_dec=PMDEC*u.mas/u.yr,
                equinox=Time(EPOCH, format='jyear'),
                obstime=Time(OBS_DATE, format='jyear', scale='tcb'),
                frame=FRAME
            )
print(acqt)

<SkyCoord (ICRS): (ra, dec) in deg
    (355.94560364, -15.27457775)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (-2.70869098, -4.08098586)>


## Update coordinates for R Aqr and acquisition star
Update position of R Aqr and acquisition star to initial and final date of observing window. Use the apply_space_motion() method of the SkyCoord package. Calculate also position of R Aqr at the time of the second observation based on the position in the first observation, as a sanity check.

In [8]:
# Update the coordinates of R Aqr to the JWST observing time
r_aqr1.apply_space_motion(new_obstime=jwst_time1)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (355.95628712, -15.28470221, 385.63881065)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (27.33000449, -30.36999596, 6.0383875e-05)>

In [7]:
from astropy.coordinates import SkyCoord, Distance
from astropy.time import Time
import astropy.units as u

# Example data for result_tgas
result_tgas = {
    'ra': 10.684,  # in degrees
    'dec': 41.269,  # in degrees
    'parallax': 0.1,  # in milliarcseconds
    'pmra': 5.0,  # in milliarcseconds/year
    'pmdec': -3.0,  # in milliarcseconds/year
    'ref_epoch': 2015.5  # in Julian years
}

# Define the SkyCoord object
c = SkyCoord(ra=result_tgas['ra'] * u.deg,
             dec=result_tgas['dec'] * u.deg,
             distance=Distance(parallax=result_tgas['parallax'] * u.mas),
             pm_ra_cosdec=result_tgas['pmra'] * u.mas/u.yr,
             pm_dec=result_tgas['pmdec'] * u.mas/u.yr,
             obstime=Time(result_tgas['ref_epoch'], format='jyear', scale='tcb'))

# Define the epoch_2mass as a Time object
epoch_2mass = Time('2000-01-01T12:00:00', format='isot', scale='utc')

# Apply space motion
try:
    c_2mass_epoch = c.apply_space_motion(new_obstime=epoch_2mass)
    print(c_2mass_epoch)
except TypeError as e:
    print(f"TypeError: {e}")

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (10.68397136, 41.26901292, 10000.00000202)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (5.00000099, -2.99999835, -0.00012112)>
